In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

### Read data

In [2]:
dataset_path = 'C:/Users/Cristian Burich/Desktop/MA/segundo/eyf/datasets/' # '/content/drive/MyDrive/Colab Notebooks/DMEyF/2024/datos/'
dataset_file = 'competencia_02_crudo.csv'

In [3]:
data = pd.read_csv(dataset_path+dataset_file)
data.head(10)

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,201901,1,0,1,59,276,7597.55,47433.58,5654.59,...,0.0,1.0,0.0,-44919.57,3.23,7136.0,24336.99,13.0,0.0,1466.25
1,249221468,201901,1,0,1,51,13,738.16,-325.09,491.31,...,0.0,1.0,0.0,-30184.75,15.24,393.0,16345.79,8.0,0.0,1548.36
2,249223005,201901,1,0,1,46,178,1014.31,9434.15,417.36,...,0.0,1.0,0.0,-3730.14,0.00,2755.0,1181.65,3.0,0.0,3917.82
3,249228180,201901,1,0,1,64,296,1028.33,6873.80,1129.43,...,0.0,1.0,0.0,-13196.25,0.00,8194.0,1892.04,1.0,0.0,2017.56
4,249232117,201901,1,0,1,77,349,11617.61,43291.55,10726.08,...,0.0,1.0,0.0,-22073.65,0.00,7876.0,21902.85,6.0,0.0,891.48
5,249236712,201901,1,0,1,42,1,1746.17,1746.17,0.00,...,NaN,1.0,0.0,NaN,NaN,10.0,NaN,NaN,NaN,0.00
6,249237079,201901,1,0,1,62,117,19193.25,110961.55,3147.21,...,0.0,8.0,0.0,-3688.58,0.00,3531.0,257.94,1.0,0.0,586.50
7,249237446,201901,1,0,1,52,253,3268.57,20250.17,220.94,...,0.0,8.0,0.0,-5574.87,0.00,3222.0,-1.34,4.0,0.0,680.34
8,249239632,201901,1,0,1,50,239,990.51,8868.52,343.85,...,0.0,1.0,0.0,-18110.05,0.00,5054.0,1596.32,2.0,0.0,1114.35
9,249242331,201901,1,0,1,53,256,3429.46,27997.33,1153.68,...,0.0,1.0,0.0,-49544.20,116.27,5708.0,35766.21,18.0,0.0,2498.49


In [4]:
%%sql

create or replace table competencia_01 as
select *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


In [5]:
%%sql
select
    foto_mes
    , count(*) as cantidad # cuenta cuantos casos hay en cada foto_mes
                           # y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,201901,124752
1,201902,125799
2,201903,126436
3,201904,126996
4,201905,127659
5,201906,129186
6,201907,130724
7,201908,132664
8,201909,134314
9,201910,136682


In [6]:
%%sql

create or replace table competencia_01 as
select *
from competencia_01
where foto_mes = 202101
or foto_mes = 202102
or foto_mes = 202103
or foto_mes = 202104
or foto_mes = 202105
or foto_mes = 202106
or foto_mes = 202107
or foto_mes = 202108

,Success


In [7]:
%%sql

create or replace table competencia_01 as

with periodos as (
    select distinct foto_mes from competencia_01
), clientes as (
    select distinct numero_de_cliente from competencia_01
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0 # genero la columna mes_0 y sobre ella genero mes_1 y mes_2
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if(mes_2 = 1, 'CONTINUA',
             if(mes_1 = 0, 'BAJA+1',
                if(mes_2 = 0,'BAJA+2',NULL))) as clase_ternaria # Replazar null por la lógica que genera el target
    from todo t
    left join competencia_01 c using (numero_de_cliente, foto_mes)
) 

select * EXCLUDE (mes_0, mes_1, mes_2) # descarto las columnas auxiliares
from clase_ternaria
where mes_0 = 1 # descarto las filas de los clientes para los meses previos a que ingresaran al banco.

,Success


In [8]:
%sql select count(*) from competencia_01


,count_star()
0,1312540


In [9]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,635,785,160606
1,202102,785,1017,160844
2,202103,1020,981,161684
3,202104,982,1189,161919
4,202105,1189,911,162523
5,202106,908,1074,162894
6,202107,1075,0,0
7,202108,0,0,0


In [10]:
%%sql 

select * from competencia_01

limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249461691,202101,1,0,0,63,320,5184.37,47486.85,2964.22,...,4,0.0,-39809.64,0.0,7552,25839.92,16.0,0.0,2123.13,CONTINUA
1,249461691,202102,1,0,0,64,321,4427.14,48887.16,2984.35,...,-3,0.0,-32646.53,0.0,7580,41661.42,20.0,0.0,2944.23,CONTINUA
2,249461691,202103,1,0,0,64,322,4935.91,50391.10,3254.86,...,1,0.0,-48657.86,0.0,7611,25120.08,15.0,0.0,2052.75,CONTINUA
3,249461691,202104,1,0,0,64,323,3801.55,50742.98,1929.76,...,2,0.0,-31233.89,0.0,7641,26204.83,15.0,0.0,2158.32,CONTINUA
4,249461691,202105,1,0,0,64,324,4234.54,51412.66,1614.34,...,5,0.0,-33222.30,0.0,7672,21115.32,15.0,0.0,1982.37,CONTINUA
5,249461691,202106,1,0,0,64,325,6527.06,54176.74,3565.12,...,0,0.0,-29974.72,0.0,7702,38276.76,17.0,0.0,2756.55,CONTINUA
6,249461691,202107,1,0,0,64,326,7083.31,57147.29,3713.74,...,3,0.0,-44887.16,0.0,7733,30133.20,13.0,0.0,2615.79,None
7,249461691,202108,1,0,0,64,327,7988.86,61521.60,3975.73,...,6,0.0,-42319.34,0.0,7764,29601.25,16.0,0.0,2803.47,None
8,251867778,202101,1,0,0,58,261,4870.53,41960.19,2450.71,...,4,0.0,NaN,NaN,5653,NaN,NaN,NaN,0.00,CONTINUA
9,251867778,202102,1,0,0,58,262,5025.90,44585.85,2448.21,...,-3,0.0,NaN,NaN,5681,NaN,NaN,NaN,0.00,CONTINUA


### Elimino variables con data drifting / concept drifting

In [11]:
%%sql
select cprestamos_personales, mprestamos_personales
from competencia_01
limit 5

,cprestamos_personales,mprestamos_personales
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [12]:
%%sql
create or replace table competencia_01 as
select * exclude(cprestamos_personales,mprestamos_personales)
from competencia_01

,Success


### Sumas de columnas

In [13]:
%%sql
create or replace table competencia_01 as
select
    *
    , mcuenta_corriente_adicional + mcuenta_corriente + mcaja_ahorro_adicional + mcaja_ahorro_dolares + mcaja_ahorro AS suma_alt # ninguna tiene null
    , cseguro_vida + cseguro_auto +	cseguro_vivienda + cseguro_accidentes_personales AS cseguro_todos # ninguna tiene null
    , ifnull(matm, 0) + ifnull(matm_other, 0) as matm_total
    
    , ifnull(Visa_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tarjeta_msaldototal
   # , ifnull(mactivos_margen, 0) +  ifnull(mpasivos_margen, 0) as map_margen
    , ifnull(ccuenta_corriente, 0) +  ifnull(ccaja_ahorro, 0) as cc_ca
    , ifnull(ctarjeta_visa, 0) +  ifnull(ctarjeta_Visa, 0) as ctarjeta_total
    , ifnull(ctarjeta_visa_transacciones, 0) +  ifnull(ctarjeta_master_transacciones, 0) as ctarjeta_total_transacciones
    , ifnull(mtarjeta_visa_consumo, 0) +  ifnull(mtarjeta_master_consumo, 0) as mtarjeta_total_consumo
    , ifnull(mplazo_fijo_dolares, 0) +  ifnull(mplazo_fijo_pesos, 0) as mplazo_fijo
    , ifnull(minversion1_pesos, 0) +  ifnull(minversion1_dolares, 0) as minversion1
    , ifnull(mpayroll, 0) +  ifnull(mpayroll2, 0) as mpayroll_todos
    , ifnull(mcuenta_debitos_automaticos, 0) +  ifnull(mtarjeta_total_consumo, 0) as consumos_mas_DA
    , ifnull(ctarjeta_visa_debitos_automaticos, 0) +  ifnull(ctarjeta_master_debitos_automaticos, 0) as ctarjeta_todas_debitos_automaticos
    , ifnull(mttarjeta_visa_debitos_automaticos, 0) +  ifnull(mttarjeta_master_debitos_automaticos, 0) as mtarjeta_todas_debitos_automaticos
    , ifnull(ctarjeta_visa_descuentos, 0) +  ifnull(ctarjeta_master_descuentos, 0) as ctarjeta_todas_descuentos
    , ifnull(mtarjeta_visa_descuentos, 0) +  ifnull(mtarjeta_master_descuentos, 0) as mtarjeta_todas_descuentos
    , ifnull(ccomisiones_mantenimiento, 0) +  ifnull(ccomisiones_otras, 0) as comisiones
    , ifnull(mcomisiones_mantenimiento, 0) +  ifnull(mcomisiones_otras, 0) as mcomisiones_suma
    , ifnull(ctransferencias_recibidas, 0) +  ifnull(ctransferencias_emitidas, 0) as ctransferencias
    , ifnull(mtransferencias_recibidas, 0) +  ifnull(mtransferencias_emitidas, 0) as mtransferencias
    , ifnull(Master_msaldopesos, 0) +  ifnull(Visa_msaldopesos, 0) as tarjetas_msaldopesos
    , ifnull(Master_msaldodolares, 0) +  ifnull(Visa_msaldodolares, 0) as tarjetas_msaldodolares
    , ifnull(Master_mconsumospesos, 0) +  ifnull(Visa_mconsumospesos, 0) as tarjetas_mconsumospesos
    , ifnull(Master_mconsumosdolares, 0) +  ifnull(Visa_mconsumosdolares, 0) as tarjetas_mconsumosdolares
    , ifnull(Master_mlimitecompra, 0) +  ifnull(Visa_mlimitecompra, 0) as tarjetas_mlimitecompra
    , ifnull(Master_madelantopesos, 0) +  ifnull(Visa_madelantopesos, 0) as tarjetas_madelantopesos
    , ifnull(Master_madelantodolares, 0) +  ifnull(Visa_madelantodolares, 0) as tarjetas_madelantodolares
    , ifnull(Master_mpagado, 0) +  ifnull(Visa_mpagado, 0) as tarjetas_mpagado
    , ifnull(Master_mpagospesos, 0) +  ifnull(Visa_mpagospesos, 0) as tarjetas_mpagospesos
    , ifnull(Master_mpagosdolares, 0) +  ifnull(Visa_mpagosdolares, 0) as tarjetas_mpagosdolares
    , ifnull(Master_mconsumototal, 0) +  ifnull(Visa_mconsumototal, 0) as tarjetas_mconsumototal
    , ifnull(Master_cconsumos, 0) +  ifnull(Visa_cconsumos, 0) as tarjetas_cconsumos
    , ifnull(Master_cadelantosefectivo, 0) +  ifnull(Visa_cadelantosefectivo, 0) as tarjetas_cadelantosefectivo
    , ifnull(Master_mpagominimo, 0) +  ifnull(Visa_mpagominimo, 0) as tarjetas_mpagominimo
    , ifnull(Master_mfinanciacion_limite, 0) +  ifnull(Visa_mfinanciacion_limite, 0) as tarjetas_mfinanciacion_limite
from competencia_01

,Success


### Promedios moviles

Variables originales

In [14]:
%%sql
create or replace table competencia_01 as
select
    *
    , avg(mrentabilidad) over ventana_3 as avg3_mrentabilidad
    , avg(mrentabilidad_annual) over ventana_3 as avg3_mrentabilidad_annual
    , avg(mcomisiones) over ventana_3 as avg3_mcomisiones
    , avg(mactivos_margen) over ventana_3 as avg3_mactivos_margen
    , avg(mpasivos_margen) over ventana_3 as avg3_mpasivos_margen
    , avg(cproductos) over ventana_3 as avg3_cproductos
    , avg(tcuentas) over ventana_3 as avg3_tcuentas
    , avg(ccuenta_corriente) over ventana_3 as avg3_ccuenta_corriente
    , avg(mcuenta_corriente_adicional) over ventana_3 as avg3_mcuenta_corriente_adicional
    , avg(mcuenta_corriente) over ventana_3 as avg3_mcuenta_corriente
    , avg(ccaja_ahorro) over ventana_3 as avg3_ccaja_ahorro
    , avg(mcaja_ahorro) over ventana_3 as avg3_mcaja_ahorro
    , avg(mcaja_ahorro_adicional) over ventana_3 as avg3_mcaja_ahorro_adicional
    , avg(mcaja_ahorro_dolares) over ventana_3 as avg3_mcaja_ahorro_dolares
    , avg(mcuentas_saldo) over ventana_3 as avg3_mcuentas_saldo
    , avg(ctarjeta_debito) over ventana_3 as avg3_ctarjeta_debito
    , avg(ctarjeta_debito_transacciones) over ventana_3 as avg3_ctarjeta_debito_transacciones
    , avg(mautoservicio) over ventana_3 as avg3_mautoservicio
    , avg(ctarjeta_visa) over ventana_3 as avg3_ctarjeta_visa
    , avg(ctarjeta_visa_transacciones) over ventana_3 as avg3_ctarjeta_visa_transacciones
    , avg(mtarjeta_visa_consumo) over ventana_3 as avg3_mtarjeta_visa_consumo
    , avg(ctarjeta_master) over ventana_3 as avg3_ctarjeta_master
    , avg(ctarjeta_master_transacciones) over ventana_3 as avg3_ctarjeta_master_transacciones
    , avg(mtarjeta_master_consumo) over ventana_3 as avg3_mtarjeta_master_consumo
    # , avg(cprestamos_personales) over ventana_3 as avg3_cprestamos_personales
    # , avg(mprestamos_personales) over ventana_3 as avg3_mprestamos_personales
    , avg(cprestamos_prendarios) over ventana_3 as avg3_cprestamos_prendarios
    , avg(mprestamos_prendarios) over ventana_3 as avg3_mprestamos_prendarios
    , avg(cprestamos_hipotecarios) over ventana_3 as avg3_cprestamos_hipotecarios
    , avg(mprestamos_hipotecarios) over ventana_3 as avg3_mprestamos_hipotecarios
    , avg(cplazo_fijo) over ventana_3 as avg3_cplazo_fijo
    , avg(mplazo_fijo_dolares) over ventana_3 as avg3_mplazo_fijo_dolares
    , avg(mplazo_fijo_pesos) over ventana_3 as avg3_mplazo_fijo_pesos
    , avg(cinversion1) over ventana_3 as avg3_cinversion1
    , avg(minversion1_pesos) over ventana_3 as avg3_minversion1_pesos
    , avg(minversion1_dolares) over ventana_3 as avg3_minversion1_dolares
    , avg(cinversion2) over ventana_3 as avg3_cinversion2
    , avg(minversion2) over ventana_3 as avg3_minversion2
    , avg(cseguro_vida) over ventana_3 as avg3_cseguro_vida
    , avg(cseguro_auto) over ventana_3 as avg3_cseguro_auto
    , avg(cseguro_vivienda) over ventana_3 as avg3_cseguro_vivienda
    , avg(cseguro_accidentes_personales) over ventana_3 as avg3_cseguro_accidentes_personales
    , avg(cpayroll_trx) over ventana_3 as avg3_cpayroll_trx
    , avg(mpayroll) over ventana_3 as avg3_mpayroll
    , avg(mpayroll2) over ventana_3 as avg3_mpayroll2
    , avg(cpayroll2_trx) over ventana_3 as avg3_cpayroll2_trx
    , avg(ccuenta_debitos_automaticos) over ventana_3 as avg3_ccuenta_debitos_automaticos
    , avg(mcuenta_debitos_automaticos) over ventana_3 as avg3_mcuenta_debitos_automaticos
    , avg(ctarjeta_visa_debitos_automaticos) over ventana_3 as avg3_ctarjeta_visa_debitos_automaticos
    , avg(mttarjeta_visa_debitos_automaticos) over ventana_3 as avg3_mttarjeta_visa_debitos_automaticos
    , avg(ctarjeta_master_debitos_automaticos) over ventana_3 as avg3_ctarjeta_master_debitos_automaticos
    , avg(mttarjeta_master_debitos_automaticos) over ventana_3 as avg3_mttarjeta_master_debitos_automaticos
    , avg(cpagodeservicios) over ventana_3 as avg3_cpagodeservicios
    , avg(mpagodeservicios) over ventana_3 as avg3_mpagodeservicios
    , avg(cpagomiscuentas) over ventana_3 as avg3_cpagomiscuentas
    , avg(mpagomiscuentas) over ventana_3 as avg3_mpagomiscuentas
    , avg(ccajeros_propios_descuentos) over ventana_3 as avg3_ccajeros_propios_descuentos
    , avg(mcajeros_propios_descuentos) over ventana_3 as avg3_mcajeros_propios_descuentos
    , avg(ctarjeta_visa_descuentos) over ventana_3 as avg3_ctarjeta_visa_descuentos
    , avg(mtarjeta_visa_descuentos) over ventana_3 as avg3_mtarjeta_visa_descuentos
    , avg(ctarjeta_master_descuentos) over ventana_3 as avg3_ctarjeta_master_descuentos
    , avg(mtarjeta_master_descuentos) over ventana_3 as avg3_mtarjeta_master_descuentos
    , avg(ccomisiones_mantenimiento) over ventana_3 as avg3_ccomisiones_mantenimiento
    , avg(mcomisiones_mantenimiento) over ventana_3 as avg3_mcomisiones_mantenimiento
    , avg(ccomisiones_otras) over ventana_3 as avg3_ccomisiones_otras
    , avg(mcomisiones_otras) over ventana_3 as avg3_mcomisiones_otras
    , avg(cforex) over ventana_3 as avg3_cforex
    , avg(cforex_buy) over ventana_3 as avg3_cforex_buy
    , avg(mforex_buy) over ventana_3 as avg3_mforex_buy
    , avg(cforex_sell) over ventana_3 as avg3_cforex_sell
    , avg(mforex_sell) over ventana_3 as avg3_mforex_sell
    , avg(ctransferencias_recibidas) over ventana_3 as avg3_ctransferencias_recibidas
    , avg(mtransferencias_recibidas) over ventana_3 as avg3_mtransferencias_recibidas
    , avg(ctransferencias_emitidas) over ventana_3 as avg3_ctransferencias_emitidas
    , avg(mtransferencias_emitidas) over ventana_3 as avg3_mtransferencias_emitidas
    , avg(cextraccion_autoservicio) over ventana_3 as avg3_cextraccion_autoservicio
    , avg(mextraccion_autoservicio) over ventana_3 as avg3_mextraccion_autoservicio
    , avg(ccheques_depositados) over ventana_3 as avg3_ccheques_depositados
    , avg(mcheques_depositados) over ventana_3 as avg3_mcheques_depositados
    , avg(ccheques_emitidos) over ventana_3 as avg3_ccheques_emitidos
    , avg(mcheques_emitidos) over ventana_3 as avg3_mcheques_emitidos
    , avg(ccheques_depositados_rechazados) over ventana_3 as avg3_ccheques_depositados_rechazados
    , avg(mcheques_depositados_rechazados) over ventana_3 as avg3_mcheques_depositados_rechazados
    , avg(ccheques_emitidos_rechazados) over ventana_3 as avg3_ccheques_emitidos_rechazados
    , avg(mcheques_emitidos_rechazados) over ventana_3 as avg3_mcheques_emitidos_rechazados
    , avg(ccallcenter_transacciones) over ventana_3 as avg3_ccallcenter_transacciones
    , avg(chomebanking_transacciones) over ventana_3 as avg3_chomebanking_transacciones
    , avg(ccajas_transacciones) over ventana_3 as avg3_ccajas_transacciones
    , avg(ccajas_consultas) over ventana_3 as avg3_ccajas_consultas
    , avg(ccajas_depositos) over ventana_3 as avg3_ccajas_depositos
    , avg(ccajas_extracciones) over ventana_3 as avg3_ccajas_extracciones
    , avg(ccajas_otras) over ventana_3 as avg3_ccajas_otras
    , avg(catm_trx) over ventana_3 as avg3_catm_trx
    , avg(matm) over ventana_3 as avg3_matm
    , avg(catm_trx_other) over ventana_3 as avg3_catm_trx_other
    , avg(matm_other) over ventana_3 as avg3_matm_other
    , avg(ctrx_quarter) over ventana_3 as avg3_ctrx_quarter
   # , avg(cmobile_app_trx) over ventana_3 as avg3_cmobile_app_trx
    , avg(Master_mfinanciacion_limite) over ventana_3 as avg3_Master_mfinanciacion_limite
    , avg(Master_msaldototal) over ventana_3 as avg3_Master_msaldototal
    , avg(Master_msaldopesos) over ventana_3 as avg3_Master_msaldopesos
    , avg(Master_msaldodolares) over ventana_3 as avg3_Master_msaldodolares
    , avg(Master_mconsumospesos) over ventana_3 as avg3_Master_mconsumospesos
    , avg(Master_mconsumosdolares) over ventana_3 as avg3_Master_mconsumosdolares
    , avg(Master_mlimitecompra) over ventana_3 as avg3_Master_mlimitecompra
    , avg(Master_madelantopesos) over ventana_3 as avg3_Master_madelantopesos
    , avg(Master_madelantodolares) over ventana_3 as avg3_Master_madelantodolares
    , avg(Master_mpagado) over ventana_3 as avg3_Master_mpagado
    , avg(Master_mpagospesos) over ventana_3 as avg3_Master_mpagospesos
    , avg(Master_mpagosdolares) over ventana_3 as avg3_Master_mpagosdolares
    , avg(Master_mconsumototal) over ventana_3 as avg3_Master_mconsumototal
    , avg(Master_cconsumos) over ventana_3 as avg3_Master_cconsumos
    , avg(Master_cadelantosefectivo) over ventana_3 as avg3_Master_cadelantosefectivo
    , avg(Master_mpagominimo) over ventana_3 as avg3_Master_mpagominimo
    , avg(Visa_mfinanciacion_limite) over ventana_3 as avg3_Visa_mfinanciacion_limite
    , avg(Visa_msaldototal) over ventana_3 as avg3_Visa_msaldototal
    , avg(Visa_msaldopesos) over ventana_3 as avg3_Visa_msaldopesos
    , avg(Visa_msaldodolares) over ventana_3 as avg3_Visa_msaldodolares
    , avg(Visa_mconsumospesos) over ventana_3 as avg3_Visa_mconsumospesos
    , avg(Visa_mconsumosdolares) over ventana_3 as avg3_Visa_mconsumosdolares
    , avg(Visa_mlimitecompra) over ventana_3 as avg3_Visa_mlimitecompra
    , avg(Visa_madelantopesos) over ventana_3 as avg3_Visa_madelantopesos
    , avg(Visa_madelantodolares) over ventana_3 as avg3_Visa_madelantodolares
    , avg(Visa_mpagado) over ventana_3 as avg3_Visa_mpagado
    , avg(Visa_mpagospesos) over ventana_3 as avg3_Visa_mpagospesos
    , avg(Visa_mpagosdolares) over ventana_3 as avg3_Visa_mpagosdolares
    , avg(Visa_mconsumototal) over ventana_3 as avg3_Visa_mconsumototal
    , avg(Visa_cconsumos) over ventana_3 as avg3_Visa_cconsumos
    , avg(Visa_cadelantosefectivo) over ventana_3 as avg3_Visa_cadelantosefectivo
    , avg(Visa_mpagominimo) over ventana_3 as avg3_Visa_mpagominimo
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

,Success


Variables creadas

In [15]:
%%sql
create or replace table competencia_01 as
select
    *
   # , avg(map_margen) over ventana_3 as avg3_map_margen
    , avg(cc_ca) over ventana_3 as avg3_cc_ca
    , avg(ctarjeta_total) over ventana_3 as avg3_ctarjeta_total
    , avg(ctarjeta_total_transacciones) over ventana_3 as avg3_ctarjeta_total_transacciones
    , avg(mtarjeta_total_consumo) over ventana_3 as avg3_mtarjeta_total_consumo
    , avg(mplazo_fijo) over ventana_3 as avg3_mplazo_fijo
    , avg(minversion1) over ventana_3 as avg3_minversion1
    , avg(cseguro_todos) over ventana_3 as avg3_cseguro_todos
    , avg(mpayroll_todos) over ventana_3 as avg3_mpayroll_todos
    , avg(consumos_mas_DA) over ventana_3 as avg3_consumos_mas_DA
    , avg(ctarjeta_todas_debitos_automaticos) over ventana_3 as avg3_ctarjeta_todas_debitos_automaticos
    , avg(mtarjeta_todas_debitos_automaticos) over ventana_3 as avg3_mtarjeta_todas_debitos_automaticos
    , avg(ctarjeta_todas_descuentos) over ventana_3 as avg3_ctarjeta_todas_descuentos
    , avg(mtarjeta_todas_descuentos) over ventana_3 as avg3_mtarjeta_todas_descuentos
    , avg(comisiones) over ventana_3 as avg3_comisiones
    , avg(mcomisiones_suma) over ventana_3 as avg3_mcomisiones_suma
    , avg(ctransferencias) over ventana_3 as avg3_ctransferencias
    , avg(mtransferencias) over ventana_3 as avg3_mtransferencias
    , avg(tarjeta_msaldototal) over ventana_3 as avg3_tarjeta_msaldototal
    , avg(tarjetas_msaldopesos) over ventana_3 as avg3_tarjetas_msaldopesos
    , avg(tarjetas_msaldodolares) over ventana_3 as avg3_tarjetas_msaldodolares
    , avg(tarjetas_mconsumospesos) over ventana_3 as avg3_tarjetas_mconsumospesos
    , avg(tarjetas_mconsumosdolares) over ventana_3 as avg3_tarjetas_mconsumosdolares
    , avg(tarjetas_mlimitecompra) over ventana_3 as avg3_tarjetas_mlimitecompra
    , avg(tarjetas_madelantopesos) over ventana_3 as avg3_tarjetas_madelantopesos
    , avg(tarjetas_madelantodolares) over ventana_3 as avg3_tarjetas_madelantodolares
    , avg(tarjetas_mpagado) over ventana_3 as avg3_tarjetas_mpagado
    , avg(tarjetas_mpagospesos) over ventana_3 as avg3_tarjetas_mpagospesos
    , avg(tarjetas_mpagosdolares) over ventana_3 as avg3_tarjetas_mpagosdolares
    , avg(tarjetas_mconsumototal) over ventana_3 as avg3_tarjetas_mconsumototal
    , avg(tarjetas_cconsumos) over ventana_3 as avg3_tarjetas_cconsumos
    , avg(tarjetas_cadelantosefectivo) over ventana_3 as avg3_tarjetas_cadelantosefectivo
    , avg(tarjetas_mpagominimo) over ventana_3 as avg3_tarjetas_mpagominimo
    , avg(tarjetas_mfinanciacion_limite) over ventana_3 as avg3_tarjetas_mfinanciacion_limite
    , avg(matm_total) over ventana_3 as avg3_matm_total
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

,Success


### Slope

Variables originales

In [16]:
%%sql
create or replace table competencia_01 as
select
    *
    , regr_slope(mrentabilidad, cliente_antiguedad) over ventana_3 as slope_mrentabilidad
    , regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_3 as slope_mrentabilidad_annual
    , regr_slope(mcomisiones, cliente_antiguedad) over ventana_3 as slope_mcomisiones
    , regr_slope(mactivos_margen, cliente_antiguedad) over ventana_3 as slope_mactivos_margen
    , regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as slope_mpasivos_margen
    , regr_slope(cproductos, cliente_antiguedad) over ventana_3 as slope_cproductos
    , regr_slope(tcuentas, cliente_antiguedad) over ventana_3 as slope_tcuentas
    , regr_slope(ccuenta_corriente, cliente_antiguedad) over ventana_3 as slope_ccuenta_corriente
    , regr_slope(mcuenta_corriente_adicional, cliente_antiguedad) over ventana_3 as slope_mcuenta_corriente_adicional
    , regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_3 as slope_mcuenta_corriente
    , regr_slope(ccaja_ahorro, cliente_antiguedad) over ventana_3 as slope_ccaja_ahorro
    , regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as slope_mcaja_ahorro
    , regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_3 as slope_mcaja_ahorro_adicional
    , regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_3 as slope_mcaja_ahorro_dolares
    , regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as slope_mcuentas_saldo
    , regr_slope(ctarjeta_debito, cliente_antiguedad) over ventana_3 as slope_ctarjeta_debito
    , regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_3 as slope_ctarjeta_debito_transacciones
    , regr_slope(mautoservicio, cliente_antiguedad) over ventana_3 as slope_mautoservicio
    , regr_slope(ctarjeta_visa, cliente_antiguedad) over ventana_3 as slope_ctarjeta_visa
    , regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_3 as slope_ctarjeta_visa_transacciones
    , regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_3 as slope_mtarjeta_visa_consumo
    , regr_slope(ctarjeta_master, cliente_antiguedad) over ventana_3 as slope_ctarjeta_master
    , regr_slope(ctarjeta_master_transacciones, cliente_antiguedad) over ventana_3 as slope_ctarjeta_master_transacciones
    , regr_slope(mtarjeta_master_consumo, cliente_antiguedad) over ventana_3 as slope_mtarjeta_master_consumo
    # , regr_slope(cprestamos_personales, cliente_antiguedad) over ventana_3 as slope_cprestamos_personales
    # , regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as slope_mprestamos_personales
    , regr_slope(cprestamos_prendarios, cliente_antiguedad) over ventana_3 as slope_cprestamos_prendarios
    , regr_slope(mprestamos_prendarios, cliente_antiguedad) over ventana_3 as slope_mprestamos_prendarios
    , regr_slope(cprestamos_hipotecarios, cliente_antiguedad) over ventana_3 as slope_cprestamos_hipotecarios
    , regr_slope(mprestamos_hipotecarios, cliente_antiguedad) over ventana_3 as slope_mprestamos_hipotecarios
    , regr_slope(cplazo_fijo, cliente_antiguedad) over ventana_3 as slope_cplazo_fijo
    , regr_slope(mplazo_fijo_dolares, cliente_antiguedad) over ventana_3 as slope_mplazo_fijo_dolares
    , regr_slope(mplazo_fijo_pesos, cliente_antiguedad) over ventana_3 as slope_mplazo_fijo_pesos
    , regr_slope(cinversion1, cliente_antiguedad) over ventana_3 as slope_cinversion1
    , regr_slope(minversion1_pesos, cliente_antiguedad) over ventana_3 as slope_minversion1_pesos
    , regr_slope(minversion1_dolares, cliente_antiguedad) over ventana_3 as slope_minversion1_dolares
    , regr_slope(cinversion2, cliente_antiguedad) over ventana_3 as slope_cinversion2
    , regr_slope(minversion2, cliente_antiguedad) over ventana_3 as slope_minversion2
    , regr_slope(cseguro_vida, cliente_antiguedad) over ventana_3 as slope_cseguro_vida
    , regr_slope(cseguro_auto, cliente_antiguedad) over ventana_3 as slope_cseguro_auto
    , regr_slope(cseguro_vivienda, cliente_antiguedad) over ventana_3 as slope_cseguro_vivienda
    , regr_slope(cseguro_accidentes_personales, cliente_antiguedad) over ventana_3 as slope_cseguro_accidentes_personales
    , regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as slope_cpayroll_trx
    , regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as slope_mpayroll
    , regr_slope(mpayroll2, cliente_antiguedad) over ventana_3 as slope_mpayroll2
    , regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_3 as slope_cpayroll2_trx
    , regr_slope(ccuenta_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_ccuenta_debitos_automaticos
    , regr_slope(mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_mcuenta_debitos_automaticos
    , regr_slope(ctarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_visa_debitos_automaticos
    , regr_slope(mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_mttarjeta_visa_debitos_automaticos
    , regr_slope(ctarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_master_debitos_automaticos
    , regr_slope(mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_mttarjeta_master_debitos_automaticos
    , regr_slope(cpagodeservicios, cliente_antiguedad) over ventana_3 as slope_cpagodeservicios
    , regr_slope(mpagodeservicios, cliente_antiguedad) over ventana_3 as slope_mpagodeservicios
    , regr_slope(cpagomiscuentas, cliente_antiguedad) over ventana_3 as slope_cpagomiscuentas
    , regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_3 as slope_mpagomiscuentas
    , regr_slope(ccajeros_propios_descuentos, cliente_antiguedad) over ventana_3 as slope_ccajeros_propios_descuentos
    , regr_slope(mcajeros_propios_descuentos, cliente_antiguedad) over ventana_3 as slope_mcajeros_propios_descuentos
    , regr_slope(ctarjeta_visa_descuentos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_visa_descuentos
    , regr_slope(mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_3 as slope_mtarjeta_visa_descuentos
    , regr_slope(ctarjeta_master_descuentos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_master_descuentos
    , regr_slope(mtarjeta_master_descuentos, cliente_antiguedad) over ventana_3 as slope_mtarjeta_master_descuentos
    , regr_slope(ccomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as slope_ccomisiones_mantenimiento
    , regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as slope_mcomisiones_mantenimiento
    , regr_slope(ccomisiones_otras, cliente_antiguedad) over ventana_3 as slope_ccomisiones_otras
    , regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_3 as slope_mcomisiones_otras
    , regr_slope(cforex, cliente_antiguedad) over ventana_3 as slope_cforex
    , regr_slope(cforex_buy, cliente_antiguedad) over ventana_3 as slope_cforex_buy
    , regr_slope(mforex_buy, cliente_antiguedad) over ventana_3 as slope_mforex_buy
    , regr_slope(cforex_sell, cliente_antiguedad) over ventana_3 as slope_cforex_sell
    , regr_slope(mforex_sell, cliente_antiguedad) over ventana_3 as slope_mforex_sell
    , regr_slope(ctransferencias_recibidas, cliente_antiguedad) over ventana_3 as slope_ctransferencias_recibidas
    , regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_3 as slope_mtransferencias_recibidas
    , regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_3 as slope_ctransferencias_emitidas
    , regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_3 as slope_mtransferencias_emitidas
    , regr_slope(cextraccion_autoservicio, cliente_antiguedad) over ventana_3 as slope_cextraccion_autoservicio
    , regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_3 as slope_mextraccion_autoservicio
    , regr_slope(ccheques_depositados, cliente_antiguedad) over ventana_3 as slope_ccheques_depositados
    , regr_slope(mcheques_depositados, cliente_antiguedad) over ventana_3 as slope_mcheques_depositados
    , regr_slope(ccheques_emitidos, cliente_antiguedad) over ventana_3 as slope_ccheques_emitidos
    , regr_slope(mcheques_emitidos, cliente_antiguedad) over ventana_3 as slope_mcheques_emitidos
    , regr_slope(ccheques_depositados_rechazados, cliente_antiguedad) over ventana_3 as slope_ccheques_depositados_rechazados
    , regr_slope(mcheques_depositados_rechazados, cliente_antiguedad) over ventana_3 as slope_mcheques_depositados_rechazados
    , regr_slope(ccheques_emitidos_rechazados, cliente_antiguedad) over ventana_3 as slope_ccheques_emitidos_rechazados
    , regr_slope(mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_3 as slope_mcheques_emitidos_rechazados
    , regr_slope(ccallcenter_transacciones, cliente_antiguedad) over ventana_3 as slope_ccallcenter_transacciones
    , regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_3 as slope_chomebanking_transacciones
    , regr_slope(ccajas_transacciones, cliente_antiguedad) over ventana_3 as slope_ccajas_transacciones
    , regr_slope(ccajas_consultas, cliente_antiguedad) over ventana_3 as slope_ccajas_consultas
    , regr_slope(ccajas_depositos, cliente_antiguedad) over ventana_3 as slope_ccajas_depositos
    , regr_slope(ccajas_extracciones, cliente_antiguedad) over ventana_3 as slope_ccajas_extracciones
    , regr_slope(ccajas_otras, cliente_antiguedad) over ventana_3 as slope_ccajas_otras
    , regr_slope(catm_trx, cliente_antiguedad) over ventana_3 as slope_catm_trx
    , regr_slope(matm, cliente_antiguedad) over ventana_3 as slope_matm
    , regr_slope(catm_trx_other, cliente_antiguedad) over ventana_3 as slope_catm_trx_other
    , regr_slope(matm_other, cliente_antiguedad) over ventana_3 as slope_matm_other
    , regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as slope_ctrx_quarter
   # , regr_slope(cmobile_app_trx, cliente_antiguedad) over ventana_3 as slope_cmobile_app_trx
    , regr_slope(Master_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as slope_Master_mfinanciacion_limite
    , regr_slope(Master_msaldototal, cliente_antiguedad) over ventana_3 as slope_Master_msaldototal
    , regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana_3 as slope_Master_msaldopesos
    , regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana_3 as slope_Master_msaldodolares
    , regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana_3 as slope_Master_mconsumospesos
    , regr_slope(Master_mconsumosdolares, cliente_antiguedad) over ventana_3 as slope_Master_mconsumosdolares
    , regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana_3 as slope_Master_mlimitecompra
    , regr_slope(Master_madelantopesos, cliente_antiguedad) over ventana_3 as slope_Master_madelantopesos
    , regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana_3 as slope_Master_madelantodolares
    , regr_slope(Master_mpagado, cliente_antiguedad) over ventana_3 as slope_Master_mpagado
    , regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana_3 as slope_Master_mpagospesos
    , regr_slope(Master_mpagosdolares, cliente_antiguedad) over ventana_3 as slope_Master_mpagosdolares
    , regr_slope(Master_mconsumototal, cliente_antiguedad) over ventana_3 as slope_Master_mconsumototal
    , regr_slope(Master_cconsumos, cliente_antiguedad) over ventana_3 as slope_Master_cconsumos
    , regr_slope(Master_cadelantosefectivo, cliente_antiguedad) over ventana_3 as slope_Master_cadelantosefectivo
    , regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana_3 as slope_Master_mpagominimo
    , regr_slope(Visa_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as slope_Visa_mfinanciacion_limite
    , regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_3 as slope_Visa_msaldototal
    , regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_3 as slope_Visa_msaldopesos
    , regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana_3 as slope_Visa_msaldodolares
    , regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana_3 as slope_Visa_mconsumospesos
    , regr_slope(Visa_mconsumosdolares, cliente_antiguedad) over ventana_3 as slope_Visa_mconsumosdolares
    , regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana_3 as slope_Visa_mlimitecompra
    , regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_3 as slope_Visa_madelantopesos
    , regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_3 as slope_Visa_madelantodolares
    , regr_slope(Visa_mpagado, cliente_antiguedad) over ventana_3 as slope_Visa_mpagado
    , regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana_3 as slope_Visa_mpagospesos
    , regr_slope(Visa_mpagosdolares, cliente_antiguedad) over ventana_3 as slope_Visa_mpagosdolares
    , regr_slope(Visa_mconsumototal, cliente_antiguedad) over ventana_3 as slope_Visa_mconsumototal
    , regr_slope(Visa_cconsumos, cliente_antiguedad) over ventana_3 as slope_Visa_cconsumos
    , regr_slope(Visa_cadelantosefectivo, cliente_antiguedad) over ventana_3 as slope_Visa_cadelantosefectivo
    , regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_3 as slope_Visa_mpagominimo
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)


,Success


Variables creadas

In [17]:
# esto no estaba comentado, lo comento yo 

# %%sql
# create or replace table competencia_01 as
# select
#     *
#    # , regr_slope(map_margen, cliente_antiguedad) over ventana_3 as slope_map_margen
#     , regr_slope(cc_ca, cliente_antiguedad) over ventana_3 as slope_cc_ca
#     , regr_slope(ctarjeta_total, cliente_antiguedad) over ventana_3 as slope_ctarjeta_total
#     , regr_slope(ctarjeta_total_transacciones, cliente_antiguedad) over ventana_3 as slope_ctarjeta_total_transacciones
#     , regr_slope(mtarjeta_total_consumo, cliente_antiguedad) over ventana_3 as slope_mtarjeta_total_consumo
#     , regr_slope(mplazo_fijo, cliente_antiguedad) over ventana_3 as slope_mplazo_fijo
#     , regr_slope(minversion1, cliente_antiguedad) over ventana_3 as slope_minversion1
#     , regr_slope(cseguro_todos, cliente_antiguedad) over ventana_3 as slope_cseguro_todos
#     , regr_slope(mpayroll_todos, cliente_antiguedad) over ventana_3 as slope_mpayroll_todos
#     , regr_slope(consumos_mas_DA, cliente_antiguedad) over ventana_3 as slope_consumos_mas_DA
#     , regr_slope(ctarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_todas_debitos_automaticos
#     , regr_slope(mtarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_mtarjeta_todas_debitos_automaticos
#     , regr_slope(ctarjeta_todas_descuentos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_todas_descuentos
#     , regr_slope(mtarjeta_todas_descuentos, cliente_antiguedad) over ventana_3 as slope_mtarjeta_todas_descuentos
#     , regr_slope(comisiones, cliente_antiguedad) over ventana_3 as slope_comisiones
#     , regr_slope(mcomisiones_suma, cliente_antiguedad) over ventana_3 as slope_mcomisiones_suma
#     , regr_slope(ctransferencias, cliente_antiguedad) over ventana_3 as slope_ctransferencias
#     , regr_slope(mtransferencias, cliente_antiguedad) over ventana_3 as slope_mtransferencias
#     , regr_slope(tarjeta_msaldototal, cliente_antiguedad) over ventana_3 as slope_tarjeta_msaldototal
#     , regr_slope(tarjetas_msaldopesos, cliente_antiguedad) over ventana_3 as slope_tarjetas_msaldopesos
#     , regr_slope(tarjetas_msaldodolares, cliente_antiguedad) over ventana_3 as slope_tarjetas_msaldodolares
#     , regr_slope(tarjetas_mconsumospesos, cliente_antiguedad) over ventana_3 as slope_tarjetas_mconsumospesos
#     , regr_slope(tarjetas_mconsumosdolares, cliente_antiguedad) over ventana_3 as slope_tarjetas_mconsumosdolares
#     , regr_slope(tarjetas_mlimitecompra, cliente_antiguedad) over ventana_3 as slope_tarjetas_mlimitecompra
#     , regr_slope(tarjetas_madelantopesos, cliente_antiguedad) over ventana_3 as slope_tarjetas_madelantopesos
#     , regr_slope(tarjetas_madelantodolares, cliente_antiguedad) over ventana_3 as slope_tarjetas_madelantodolares
#     , regr_slope(tarjetas_mpagado, cliente_antiguedad) over ventana_3 as slope_tarjetas_mpagado
#     , regr_slope(tarjetas_mpagospesos, cliente_antiguedad) over ventana_3 as slope_tarjetas_mpagospesos
#     , regr_slope(tarjetas_mpagosdolares, cliente_antiguedad) over ventana_3 as slope_tarjetas_mpagosdolares
#     , regr_slope(tarjetas_mconsumototal, cliente_antiguedad) over ventana_3 as slope_tarjetas_mconsumototal
#     , regr_slope(tarjetas_cconsumos, cliente_antiguedad) over ventana_3 as slope_tarjetas_cconsumos
#     , regr_slope(tarjetas_cadelantosefectivo, cliente_antiguedad) over ventana_3 as slope_tarjetas_cadelantosefectivo
#     , regr_slope(tarjetas_mpagominimo, cliente_antiguedad) over ventana_3 as slope_tarjetas_mpagominimo
#     , regr_slope(tarjetas_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as slope_tarjetas_mfinanciacion_limite
#     , regr_slope(matm_total, cliente_antiguedad) over ventana_3 as slope_matm_total
#     , regr_slope(avg3_mrentabilidad, cliente_antiguedad) over ventana_3 as slope_avg3_mrentabilidad
#     , regr_slope(avg3_mrentabilidad_annual, cliente_antiguedad) over ventana_3 as slope_avg3_mrentabilidad_annual
#     , regr_slope(avg3_mcomisiones, cliente_antiguedad) over ventana_3 as slope_avg3_mcomisiones
#     , regr_slope(avg3_mactivos_margen, cliente_antiguedad) over ventana_3 as slope_avg3_mactivos_margen
#     , regr_slope(avg3_mpasivos_margen, cliente_antiguedad) over ventana_3 as slope_avg3_mpasivos_margen
#     , regr_slope(avg3_cproductos, cliente_antiguedad) over ventana_3 as slope_avg3_cproductos
#     , regr_slope(avg3_tcuentas, cliente_antiguedad) over ventana_3 as slope_avg3_tcuentas
#     , regr_slope(avg3_ccuenta_corriente, cliente_antiguedad) over ventana_3 as slope_avg3_ccuenta_corriente
#     , regr_slope(avg3_mcuenta_corriente_adicional, cliente_antiguedad) over ventana_3 as slope_avg3_mcuenta_corriente_adicional
#     , regr_slope(avg3_mcuenta_corriente, cliente_antiguedad) over ventana_3 as slope_avg3_mcuenta_corriente
#     , regr_slope(avg3_ccaja_ahorro, cliente_antiguedad) over ventana_3 as slope_avg3_ccaja_ahorro
#     , regr_slope(avg3_mcaja_ahorro, cliente_antiguedad) over ventana_3 as slope_avg3_mcaja_ahorro
#     , regr_slope(avg3_mcaja_ahorro_adicional, cliente_antiguedad) over ventana_3 as slope_avg3_mcaja_ahorro_adicional
#     , regr_slope(avg3_mcaja_ahorro_dolares, cliente_antiguedad) over ventana_3 as slope_avg3_mcaja_ahorro_dolares
#     , regr_slope(avg3_mcuentas_saldo, cliente_antiguedad) over ventana_3 as slope_avg3_mcuentas_saldo
#     , regr_slope(avg3_ctarjeta_debito, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_debito
#     , regr_slope(avg3_ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_debito_transacciones
#     , regr_slope(avg3_mautoservicio, cliente_antiguedad) over ventana_3 as slope_avg3_mautoservicio
#     , regr_slope(avg3_ctarjeta_visa, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_visa
#     , regr_slope(avg3_ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_visa_transacciones
#     , regr_slope(avg3_mtarjeta_visa_consumo, cliente_antiguedad) over ventana_3 as slope_avg3_mtarjeta_visa_consumo
#     , regr_slope(avg3_ctarjeta_master, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_master
#     , regr_slope(avg3_ctarjeta_master_transacciones, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_master_transacciones
#     , regr_slope(avg3_mtarjeta_master_consumo, cliente_antiguedad) over ventana_3 as slope_avg3_mtarjeta_master_consumo
#     #, regr_slope(avg3_cprestamos_personales, cliente_antiguedad) over ventana_3 as slope_avg3_cprestamos_personales
#     #, regr_slope(avg3_mprestamos_personales, cliente_antiguedad) over ventana_3 as slope_avg3_mprestamos_personales
#     , regr_slope(avg3_cprestamos_prendarios, cliente_antiguedad) over ventana_3 as slope_avg3_cprestamos_prendarios
#     , regr_slope(avg3_mprestamos_prendarios, cliente_antiguedad) over ventana_3 as slope_avg3_mprestamos_prendarios
#     , regr_slope(avg3_cprestamos_hipotecarios, cliente_antiguedad) over ventana_3 as slope_avg3_cprestamos_hipotecarios
#     , regr_slope(avg3_mprestamos_hipotecarios, cliente_antiguedad) over ventana_3 as slope_avg3_mprestamos_hipotecarios
#     , regr_slope(avg3_cplazo_fijo, cliente_antiguedad) over ventana_3 as slope_avg3_cplazo_fijo
#     , regr_slope(avg3_mplazo_fijo_dolares, cliente_antiguedad) over ventana_3 as slope_avg3_mplazo_fijo_dolares
#     , regr_slope(avg3_mplazo_fijo_pesos, cliente_antiguedad) over ventana_3 as slope_avg3_mplazo_fijo_pesos
#     , regr_slope(avg3_cinversion1, cliente_antiguedad) over ventana_3 as slope_avg3_cinversion1
#     , regr_slope(avg3_minversion1_pesos, cliente_antiguedad) over ventana_3 as slope_avg3_minversion1_pesos
#     , regr_slope(avg3_minversion1_dolares, cliente_antiguedad) over ventana_3 as slope_avg3_minversion1_dolares
#     , regr_slope(avg3_cinversion2, cliente_antiguedad) over ventana_3 as slope_avg3_cinversion2
#     , regr_slope(avg3_minversion2, cliente_antiguedad) over ventana_3 as slope_avg3_minversion2
#     , regr_slope(avg3_cseguro_vida, cliente_antiguedad) over ventana_3 as slope_avg3_cseguro_vida
#     , regr_slope(avg3_cseguro_auto, cliente_antiguedad) over ventana_3 as slope_avg3_cseguro_auto
#     , regr_slope(avg3_cseguro_vivienda, cliente_antiguedad) over ventana_3 as slope_avg3_cseguro_vivienda
#     , regr_slope(avg3_cseguro_accidentes_personales, cliente_antiguedad) over ventana_3 as slope_avg3_cseguro_accidentes_personales
#     , regr_slope(avg3_cpayroll_trx, cliente_antiguedad) over ventana_3 as slope_avg3_cpayroll_trx
#     , regr_slope(avg3_mpayroll, cliente_antiguedad) over ventana_3 as slope_avg3_mpayroll
#     , regr_slope(avg3_mpayroll2, cliente_antiguedad) over ventana_3 as slope_avg3_mpayroll2
#     , regr_slope(avg3_cpayroll2_trx, cliente_antiguedad) over ventana_3 as slope_avg3_cpayroll2_trx
#     , regr_slope(avg3_ccuenta_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_avg3_ccuenta_debitos_automaticos
#     , regr_slope(avg3_mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_avg3_mcuenta_debitos_automaticos
#     , regr_slope(avg3_ctarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_visa_debitos_automaticos
#     , regr_slope(avg3_mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_avg3_mttarjeta_visa_debitos_automaticos
#     , regr_slope(avg3_ctarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_master_debitos_automaticos
#     , regr_slope(avg3_mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_avg3_mttarjeta_master_debitos_automaticos
#     , regr_slope(avg3_cpagodeservicios, cliente_antiguedad) over ventana_3 as slope_avg3_cpagodeservicios
#     , regr_slope(avg3_mpagodeservicios, cliente_antiguedad) over ventana_3 as slope_avg3_mpagodeservicios
#     , regr_slope(avg3_cpagomiscuentas, cliente_antiguedad) over ventana_3 as slope_avg3_cpagomiscuentas
#     , regr_slope(avg3_mpagomiscuentas, cliente_antiguedad) over ventana_3 as slope_avg3_mpagomiscuentas
#     , regr_slope(avg3_ccajeros_propios_descuentos, cliente_antiguedad) over ventana_3 as slope_avg3_ccajeros_propios_descuentos
#     , regr_slope(avg3_mcajeros_propios_descuentos, cliente_antiguedad) over ventana_3 as slope_avg3_mcajeros_propios_descuentos
#     , regr_slope(avg3_ctarjeta_visa_descuentos, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_visa_descuentos
#     , regr_slope(avg3_mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_3 as slope_avg3_mtarjeta_visa_descuentos
#     , regr_slope(avg3_ctarjeta_master_descuentos, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_master_descuentos
#     , regr_slope(avg3_mtarjeta_master_descuentos, cliente_antiguedad) over ventana_3 as slope_avg3_mtarjeta_master_descuentos
#     , regr_slope(avg3_ccomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as slope_avg3_ccomisiones_mantenimiento
#     , regr_slope(avg3_mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as slope_avg3_mcomisiones_mantenimiento
#     , regr_slope(avg3_ccomisiones_otras, cliente_antiguedad) over ventana_3 as slope_avg3_ccomisiones_otras
#     , regr_slope(avg3_mcomisiones_otras, cliente_antiguedad) over ventana_3 as slope_avg3_mcomisiones_otras
#     , regr_slope(avg3_cforex, cliente_antiguedad) over ventana_3 as slope_avg3_cforex
#     , regr_slope(avg3_cforex_buy, cliente_antiguedad) over ventana_3 as slope_avg3_cforex_buy
#     , regr_slope(avg3_mforex_buy, cliente_antiguedad) over ventana_3 as slope_avg3_mforex_buy
#     , regr_slope(avg3_cforex_sell, cliente_antiguedad) over ventana_3 as slope_avg3_cforex_sell
#     , regr_slope(avg3_mforex_sell, cliente_antiguedad) over ventana_3 as slope_avg3_mforex_sell
#     , regr_slope(avg3_ctransferencias_recibidas, cliente_antiguedad) over ventana_3 as slope_avg3_ctransferencias_recibidas
#     , regr_slope(avg3_mtransferencias_recibidas, cliente_antiguedad) over ventana_3 as slope_avg3_mtransferencias_recibidas
#     , regr_slope(avg3_ctransferencias_emitidas, cliente_antiguedad) over ventana_3 as slope_avg3_ctransferencias_emitidas
#     , regr_slope(avg3_mtransferencias_emitidas, cliente_antiguedad) over ventana_3 as slope_avg3_mtransferencias_emitidas
#     , regr_slope(avg3_cextraccion_autoservicio, cliente_antiguedad) over ventana_3 as slope_avg3_cextraccion_autoservicio
#     , regr_slope(avg3_mextraccion_autoservicio, cliente_antiguedad) over ventana_3 as slope_avg3_mextraccion_autoservicio
#     , regr_slope(avg3_ccheques_depositados, cliente_antiguedad) over ventana_3 as slope_avg3_ccheques_depositados
#     , regr_slope(avg3_mcheques_depositados, cliente_antiguedad) over ventana_3 as slope_avg3_mcheques_depositados
#     , regr_slope(avg3_ccheques_emitidos, cliente_antiguedad) over ventana_3 as slope_avg3_ccheques_emitidos
#     , regr_slope(avg3_mcheques_emitidos, cliente_antiguedad) over ventana_3 as slope_avg3_mcheques_emitidos
#     , regr_slope(avg3_ccheques_depositados_rechazados, cliente_antiguedad) over ventana_3 as slope_avg3_ccheques_depositados_rechazados
#     , regr_slope(avg3_mcheques_depositados_rechazados, cliente_antiguedad) over ventana_3 as slope_avg3_mcheques_depositados_rechazados
#     , regr_slope(avg3_ccheques_emitidos_rechazados, cliente_antiguedad) over ventana_3 as slope_avg3_ccheques_emitidos_rechazados
#     , regr_slope(avg3_mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_3 as slope_avg3_mcheques_emitidos_rechazados
#     , regr_slope(avg3_ccallcenter_transacciones, cliente_antiguedad) over ventana_3 as slope_avg3_ccallcenter_transacciones
#     , regr_slope(avg3_chomebanking_transacciones, cliente_antiguedad) over ventana_3 as slope_avg3_chomebanking_transacciones
#     , regr_slope(avg3_ccajas_transacciones, cliente_antiguedad) over ventana_3 as slope_avg3_ccajas_transacciones
#     , regr_slope(avg3_ccajas_consultas, cliente_antiguedad) over ventana_3 as slope_avg3_ccajas_consultas
#     , regr_slope(avg3_ccajas_depositos, cliente_antiguedad) over ventana_3 as slope_avg3_ccajas_depositos
#     , regr_slope(avg3_ccajas_extracciones, cliente_antiguedad) over ventana_3 as slope_avg3_ccajas_extracciones
#     , regr_slope(avg3_ccajas_otras, cliente_antiguedad) over ventana_3 as slope_avg3_ccajas_otras
#     , regr_slope(avg3_catm_trx, cliente_antiguedad) over ventana_3 as slope_avg3_catm_trx
#     , regr_slope(avg3_matm, cliente_antiguedad) over ventana_3 as slope_avg3_matm
#     , regr_slope(avg3_catm_trx_other, cliente_antiguedad) over ventana_3 as slope_avg3_catm_trx_other
#     , regr_slope(avg3_matm_other, cliente_antiguedad) over ventana_3 as slope_avg3_matm_other
#     , regr_slope(avg3_ctrx_quarter, cliente_antiguedad) over ventana_3 as slope_avg3_ctrx_quarter
#     #, regr_slope(avg3_cmobile_app_trx, cliente_antiguedad) over ventana_3 as slope_avg3_cmobile_app_trx
#     , regr_slope(avg3_Master_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as slope_avg3_Master_mfinanciacion_limite
#     , regr_slope(avg3_Master_msaldototal, cliente_antiguedad) over ventana_3 as slope_avg3_Master_msaldototal
#     , regr_slope(avg3_Master_msaldopesos, cliente_antiguedad) over ventana_3 as slope_avg3_Master_msaldopesos
#     , regr_slope(avg3_Master_msaldodolares, cliente_antiguedad) over ventana_3 as slope_avg3_Master_msaldodolares
#     , regr_slope(avg3_Master_mconsumospesos, cliente_antiguedad) over ventana_3 as slope_avg3_Master_mconsumospesos
#     , regr_slope(avg3_Master_mconsumosdolares, cliente_antiguedad) over ventana_3 as slope_avg3_Master_mconsumosdolares
#     , regr_slope(avg3_Master_mlimitecompra, cliente_antiguedad) over ventana_3 as slope_avg3_Master_mlimitecompra
#     , regr_slope(avg3_Master_madelantopesos, cliente_antiguedad) over ventana_3 as slope_avg3_Master_madelantopesos
#     , regr_slope(avg3_Master_madelantodolares, cliente_antiguedad) over ventana_3 as slope_avg3_Master_madelantodolares
#     , regr_slope(avg3_Master_mpagado, cliente_antiguedad) over ventana_3 as slope_avg3_Master_mpagado
#     , regr_slope(avg3_Master_mpagospesos, cliente_antiguedad) over ventana_3 as slope_avg3_Master_mpagospesos
#     , regr_slope(avg3_Master_mpagosdolares, cliente_antiguedad) over ventana_3 as slope_avg3_Master_mpagosdolares
#     , regr_slope(avg3_Master_mconsumototal, cliente_antiguedad) over ventana_3 as slope_avg3_Master_mconsumototal
#     , regr_slope(avg3_Master_cconsumos, cliente_antiguedad) over ventana_3 as slope_avg3_Master_cconsumos
#     , regr_slope(avg3_Master_cadelantosefectivo, cliente_antiguedad) over ventana_3 as slope_avg3_Master_cadelantosefectivo
#     , regr_slope(avg3_Master_mpagominimo, cliente_antiguedad) over ventana_3 as slope_avg3_Master_mpagominimo
#     , regr_slope(avg3_Visa_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_mfinanciacion_limite
#     , regr_slope(avg3_Visa_msaldototal, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_msaldototal
#     , regr_slope(avg3_Visa_msaldopesos, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_msaldopesos
#     , regr_slope(avg3_Visa_msaldodolares, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_msaldodolares
#     , regr_slope(avg3_Visa_mconsumospesos, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_mconsumospesos
#     , regr_slope(avg3_Visa_mconsumosdolares, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_mconsumosdolares
#     , regr_slope(avg3_Visa_mlimitecompra, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_mlimitecompra
#     , regr_slope(avg3_Visa_madelantopesos, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_madelantopesos
#     , regr_slope(avg3_Visa_madelantodolares, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_madelantodolares
#     , regr_slope(avg3_Visa_mpagado, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_mpagado
#     , regr_slope(avg3_Visa_mpagospesos, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_mpagospesos
#     , regr_slope(avg3_Visa_mpagosdolares, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_mpagosdolares
#     , regr_slope(avg3_Visa_mconsumototal, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_mconsumototal
#     , regr_slope(avg3_Visa_cconsumos, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_cconsumos
#     , regr_slope(avg3_Visa_cadelantosefectivo, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_cadelantosefectivo
#     , regr_slope(avg3_Visa_mpagominimo, cliente_antiguedad) over ventana_3 as slope_avg3_Visa_mpagominimo
#   #  , regr_slope(avg3_map_margen, cliente_antiguedad) over ventana_3 as slope_avg3_map_margen
#     , regr_slope(avg3_cc_ca, cliente_antiguedad) over ventana_3 as slope_avg3_cc_ca
#     , regr_slope(avg3_ctarjeta_total, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_total
#     , regr_slope(avg3_ctarjeta_total_transacciones, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_total_transacciones
#     , regr_slope(avg3_mtarjeta_total_consumo, cliente_antiguedad) over ventana_3 as slope_avg3_mtarjeta_total_consumo
#     , regr_slope(avg3_mplazo_fijo, cliente_antiguedad) over ventana_3 as slope_avg3_mplazo_fijo
#     , regr_slope(avg3_minversion1, cliente_antiguedad) over ventana_3 as slope_avg3_minversion1
#     , regr_slope(avg3_cseguro_todos, cliente_antiguedad) over ventana_3 as slope_avg3_cseguro_todos
#     , regr_slope(avg3_mpayroll_todos, cliente_antiguedad) over ventana_3 as slope_avg3_mpayroll_todos
#     , regr_slope(avg3_consumos_mas_DA, cliente_antiguedad) over ventana_3 as slope_avg3_consumos_mas_DA
#     , regr_slope(avg3_ctarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_todas_debitos_automaticos
#     , regr_slope(avg3_mtarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_avg3_mtarjeta_todas_debitos_automaticos
#     , regr_slope(avg3_ctarjeta_todas_descuentos, cliente_antiguedad) over ventana_3 as slope_avg3_ctarjeta_todas_descuentos
#     , regr_slope(avg3_mtarjeta_todas_descuentos, cliente_antiguedad) over ventana_3 as slope_avg3_mtarjeta_todas_descuentos
#     , regr_slope(avg3_comisiones, cliente_antiguedad) over ventana_3 as slope_avg3_comisiones
#     , regr_slope(avg3_mcomisiones_suma, cliente_antiguedad) over ventana_3 as slope_avg3_mcomisiones_suma
#     , regr_slope(avg3_ctransferencias, cliente_antiguedad) over ventana_3 as slope_avg3_ctransferencias
#     , regr_slope(avg3_mtransferencias, cliente_antiguedad) over ventana_3 as slope_avg3_mtransferencias
#     , regr_slope(avg3_tarjeta_msaldototal, cliente_antiguedad) over ventana_3 as slope_avg3_tarjeta_msaldototal
#     , regr_slope(avg3_tarjetas_msaldopesos, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_msaldopesos
#     , regr_slope(avg3_tarjetas_msaldodolares, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_msaldodolares
#     , regr_slope(avg3_tarjetas_mconsumospesos, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_mconsumospesos
#     , regr_slope(avg3_tarjetas_mconsumosdolares, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_mconsumosdolares
#     , regr_slope(avg3_tarjetas_mlimitecompra, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_mlimitecompra
#     , regr_slope(avg3_tarjetas_madelantopesos, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_madelantopesos
#     , regr_slope(avg3_tarjetas_madelantodolares, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_madelantodolares
#     , regr_slope(avg3_tarjetas_mpagado, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_mpagado
#     , regr_slope(avg3_tarjetas_mpagospesos, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_mpagospesos
#     , regr_slope(avg3_tarjetas_mpagosdolares, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_mpagosdolares
#     , regr_slope(avg3_tarjetas_mconsumototal, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_mconsumototal
#     , regr_slope(avg3_tarjetas_cconsumos, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_cconsumos
#     , regr_slope(avg3_tarjetas_cadelantosefectivo, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_cadelantosefectivo
#     , regr_slope(avg3_tarjetas_mpagominimo, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_mpagominimo
#     , regr_slope(avg3_tarjetas_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as slope_avg3_tarjetas_mfinanciacion_limite
#     , regr_slope(avg3_matm_total, cliente_antiguedad) over ventana_3 as slope_avg3_matm_total
# from competencia_01
# window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

### Comparison with the MA

In [18]:
# %%sql
# create or replace table competencia_01 as
# select
#     *
#     , if(mrentabilidad > avg3_mrentabilidad, 1, if(mrentabilidad = avg3_mrentabilidad, 0, -1)) as mrentabilidad_comp
#     , if(mrentabilidad_annual > avg3_mrentabilidad_annual, 1, if(mrentabilidad_annual = avg3_mrentabilidad_annual, 0, -1)) as mrentabilidad_annual_comp
#     , if(mcomisiones > avg3_mcomisiones, 1, if(mcomisiones = avg3_mcomisiones, 0, -1)) as mcomisiones_comp
#     , if(mactivos_margen > avg3_mactivos_margen, 1, if(mactivos_margen = avg3_mactivos_margen, 0, -1)) as mactivos_margen_comp
#     , if(mpasivos_margen > avg3_mpasivos_margen, 1, if(mpasivos_margen = avg3_mpasivos_margen, 0, -1)) as mpasivos_margen_comp
#     , if(cproductos > avg3_cproductos, 1, if(cproductos = avg3_cproductos, 0, -1)) as cproductos_comp
#     , if(tcuentas > avg3_tcuentas, 1, if(tcuentas = avg3_tcuentas, 0, -1)) as tcuentas_comp
#     , if(ccuenta_corriente > avg3_ccuenta_corriente, 1, if(ccuenta_corriente = avg3_ccuenta_corriente, 0, -1)) as ccuenta_corriente_comp
#     , if(mcuenta_corriente_adicional > avg3_mcuenta_corriente_adicional, 1, if(mcuenta_corriente_adicional = avg3_mcuenta_corriente_adicional, 0, -1)) as mcuenta_corriente_adicional_comp
#     , if(mcuenta_corriente > avg3_mcuenta_corriente, 1, if(mcuenta_corriente = avg3_mcuenta_corriente, 0, -1)) as mcuenta_corriente_comp
#     , if(ccaja_ahorro > avg3_ccaja_ahorro, 1, if(ccaja_ahorro = avg3_ccaja_ahorro, 0, -1)) as ccaja_ahorro_comp
#     , if(mcaja_ahorro > avg3_mcaja_ahorro, 1, if(mcaja_ahorro = avg3_mcaja_ahorro, 0, -1)) as mcaja_ahorro_comp
#     , if(mcaja_ahorro_adicional > avg3_mcaja_ahorro_adicional, 1, if(mcaja_ahorro_adicional = avg3_mcaja_ahorro_adicional, 0, -1)) as mcaja_ahorro_adicional_comp
#     , if(mcaja_ahorro_dolares > avg3_mcaja_ahorro_dolares, 1, if(mcaja_ahorro_dolares = avg3_mcaja_ahorro_dolares, 0, -1)) as mcaja_ahorro_dolares_comp
#     , if(mcuentas_saldo > avg3_mcuentas_saldo, 1, if(mcuentas_saldo = avg3_mcuentas_saldo, 0, -1)) as mcuentas_saldo_comp
#     , if(ctarjeta_debito > avg3_ctarjeta_debito, 1, if(ctarjeta_debito = avg3_ctarjeta_debito, 0, -1)) as ctarjeta_debito_comp
#     , if(ctarjeta_debito_transacciones > avg3_ctarjeta_debito_transacciones, 1, if(ctarjeta_debito_transacciones = avg3_ctarjeta_debito_transacciones, 0, -1)) as ctarjeta_debito_transacciones_comp
#     , if(mautoservicio > avg3_mautoservicio, 1, if(mautoservicio = avg3_mautoservicio, 0, -1)) as mautoservicio_comp
#     , if(ctarjeta_visa > avg3_ctarjeta_visa, 1, if(ctarjeta_visa = avg3_ctarjeta_visa, 0, -1)) as ctarjeta_visa_comp
#     , if(ctarjeta_visa_transacciones > avg3_ctarjeta_visa_transacciones, 1, if(ctarjeta_visa_transacciones = avg3_ctarjeta_visa_transacciones, 0, -1)) as ctarjeta_visa_transacciones_comp
#     , if(mtarjeta_visa_consumo > avg3_mtarjeta_visa_consumo, 1, if(mtarjeta_visa_consumo = avg3_mtarjeta_visa_consumo, 0, -1)) as mtarjeta_visa_consumo_comp
#     , if(ctarjeta_master > avg3_ctarjeta_master, 1, if(ctarjeta_master = avg3_ctarjeta_master, 0, -1)) as ctarjeta_master_comp
#     , if(ctarjeta_master_transacciones > avg3_ctarjeta_master_transacciones, 1, if(ctarjeta_master_transacciones = avg3_ctarjeta_master_transacciones, 0, -1)) as ctarjeta_master_transacciones_comp
#     , if(mtarjeta_master_consumo > avg3_mtarjeta_master_consumo, 1, if(mtarjeta_master_consumo = avg3_mtarjeta_master_consumo, 0, -1)) as mtarjeta_master_consumo_comp
#     # , if(cprestamos_personales > avg3_cprestamos_personales, 1, if(cprestamos_personales = avg3_cprestamos_personales, 0, -1)) as cprestamos_personales_comp
#     # , if(mprestamos_personales > avg3_mprestamos_personales, 1, if(mprestamos_personales = avg3_mprestamos_personales, 0, -1)) as mprestamos_personales_comp
#     , if(cprestamos_prendarios > avg3_cprestamos_prendarios, 1, if(cprestamos_prendarios = avg3_cprestamos_prendarios, 0, -1)) as cprestamos_prendarios_comp
#     , if(mprestamos_prendarios > avg3_mprestamos_prendarios, 1, if(mprestamos_prendarios = avg3_mprestamos_prendarios, 0, -1)) as mprestamos_prendarios_comp
#     , if(cprestamos_hipotecarios > avg3_cprestamos_hipotecarios, 1, if(cprestamos_hipotecarios = avg3_cprestamos_hipotecarios, 0, -1)) as cprestamos_hipotecarios_comp
#     , if(mprestamos_hipotecarios > avg3_mprestamos_hipotecarios, 1, if(mprestamos_hipotecarios = avg3_mprestamos_hipotecarios, 0, -1)) as mprestamos_hipotecarios_comp
#     , if(cplazo_fijo > avg3_cplazo_fijo, 1, if(cplazo_fijo = avg3_cplazo_fijo, 0, -1)) as cplazo_fijo_comp
#     , if(mplazo_fijo_dolares > avg3_mplazo_fijo_dolares, 1, if(mplazo_fijo_dolares = avg3_mplazo_fijo_dolares, 0, -1)) as mplazo_fijo_dolares_comp
#     , if(mplazo_fijo_pesos > avg3_mplazo_fijo_pesos, 1, if(mplazo_fijo_pesos = avg3_mplazo_fijo_pesos, 0, -1)) as mplazo_fijo_pesos_comp
#     , if(cinversion1 > avg3_cinversion1, 1, if(cinversion1 = avg3_cinversion1, 0, -1)) as cinversion1_comp
#     , if(minversion1_pesos > avg3_minversion1_pesos, 1, if(minversion1_pesos = avg3_minversion1_pesos, 0, -1)) as minversion1_pesos_comp
#     , if(minversion1_dolares > avg3_minversion1_dolares, 1, if(minversion1_dolares = avg3_minversion1_dolares, 0, -1)) as minversion1_dolares_comp
#     , if(cinversion2 > avg3_cinversion2, 1, if(cinversion2 = avg3_cinversion2, 0, -1)) as cinversion2_comp
#     , if(minversion2 > avg3_minversion2, 1, if(minversion2 = avg3_minversion2, 0, -1)) as minversion2_comp
#     , if(cseguro_vida > avg3_cseguro_vida, 1, if(cseguro_vida = avg3_cseguro_vida, 0, -1)) as cseguro_vida_comp
#     , if(cseguro_auto > avg3_cseguro_auto, 1, if(cseguro_auto = avg3_cseguro_auto, 0, -1)) as cseguro_auto_comp
#     , if(cseguro_vivienda > avg3_cseguro_vivienda, 1, if(cseguro_vivienda = avg3_cseguro_vivienda, 0, -1)) as cseguro_vivienda_comp
#     , if(cseguro_accidentes_personales > avg3_cseguro_accidentes_personales, 1, if(cseguro_accidentes_personales = avg3_cseguro_accidentes_personales, 0, -1)) as cseguro_accidentes_personales_comp
#     , if(cpayroll_trx > avg3_cpayroll_trx, 1, if(cpayroll_trx = avg3_cpayroll_trx, 0, -1)) as cpayroll_trx_comp
#     , if(mpayroll > avg3_mpayroll, 1, if(mpayroll = avg3_mpayroll, 0, -1)) as mpayroll_comp
#     , if(mpayroll2 > avg3_mpayroll2, 1, if(mpayroll2 = avg3_mpayroll2, 0, -1)) as mpayroll2_comp
#     , if(cpayroll2_trx > avg3_cpayroll2_trx, 1, if(cpayroll2_trx = avg3_cpayroll2_trx, 0, -1)) as cpayroll2_trx_comp
#     , if(ccuenta_debitos_automaticos > avg3_ccuenta_debitos_automaticos, 1, if(ccuenta_debitos_automaticos = avg3_ccuenta_debitos_automaticos, 0, -1)) as ccuenta_debitos_automaticos_comp
#     , if(mcuenta_debitos_automaticos > avg3_mcuenta_debitos_automaticos, 1, if(mcuenta_debitos_automaticos = avg3_mcuenta_debitos_automaticos, 0, -1)) as mcuenta_debitos_automaticos_comp
#     , if(ctarjeta_visa_debitos_automaticos > avg3_ctarjeta_visa_debitos_automaticos, 1, if(ctarjeta_visa_debitos_automaticos = avg3_ctarjeta_visa_debitos_automaticos, 0, -1)) as ctarjeta_visa_debitos_automaticos_comp
#     , if(mttarjeta_visa_debitos_automaticos > avg3_mttarjeta_visa_debitos_automaticos, 1, if(mttarjeta_visa_debitos_automaticos = avg3_mttarjeta_visa_debitos_automaticos, 0, -1)) as mttarjeta_visa_debitos_automaticos_comp
#     , if(ctarjeta_master_debitos_automaticos > avg3_ctarjeta_master_debitos_automaticos, 1, if(ctarjeta_master_debitos_automaticos = avg3_ctarjeta_master_debitos_automaticos, 0, -1)) as ctarjeta_master_debitos_automaticos_comp
#     , if(mttarjeta_master_debitos_automaticos > avg3_mttarjeta_master_debitos_automaticos, 1, if(mttarjeta_master_debitos_automaticos = avg3_mttarjeta_master_debitos_automaticos, 0, -1)) as mttarjeta_master_debitos_automaticos_comp
#     , if(cpagodeservicios > avg3_cpagodeservicios, 1, if(cpagodeservicios = avg3_cpagodeservicios, 0, -1)) as cpagodeservicios_comp
#     , if(mpagodeservicios > avg3_mpagodeservicios, 1, if(mpagodeservicios = avg3_mpagodeservicios, 0, -1)) as mpagodeservicios_comp
#     , if(cpagomiscuentas > avg3_cpagomiscuentas, 1, if(cpagomiscuentas = avg3_cpagomiscuentas, 0, -1)) as cpagomiscuentas_comp
#     , if(mpagomiscuentas > avg3_mpagomiscuentas, 1, if(mpagomiscuentas = avg3_mpagomiscuentas, 0, -1)) as mpagomiscuentas_comp
#     , if(ccajeros_propios_descuentos > avg3_ccajeros_propios_descuentos, 1, if(ccajeros_propios_descuentos = avg3_ccajeros_propios_descuentos, 0, -1)) as ccajeros_propios_descuentos_comp
#     , if(mcajeros_propios_descuentos > avg3_mcajeros_propios_descuentos, 1, if(mcajeros_propios_descuentos = avg3_mcajeros_propios_descuentos, 0, -1)) as mcajeros_propios_descuentos_comp
#     , if(ctarjeta_visa_descuentos > avg3_ctarjeta_visa_descuentos, 1, if(ctarjeta_visa_descuentos = avg3_ctarjeta_visa_descuentos, 0, -1)) as ctarjeta_visa_descuentos_comp
#     , if(mtarjeta_visa_descuentos > avg3_mtarjeta_visa_descuentos, 1, if(mtarjeta_visa_descuentos = avg3_mtarjeta_visa_descuentos, 0, -1)) as mtarjeta_visa_descuentos_comp
#     , if(ctarjeta_master_descuentos > avg3_ctarjeta_master_descuentos, 1, if(ctarjeta_master_descuentos = avg3_ctarjeta_master_descuentos, 0, -1)) as ctarjeta_master_descuentos_comp
#     , if(mtarjeta_master_descuentos > avg3_mtarjeta_master_descuentos, 1, if(mtarjeta_master_descuentos = avg3_mtarjeta_master_descuentos, 0, -1)) as mtarjeta_master_descuentos_comp
#     , if(ccomisiones_mantenimiento > avg3_ccomisiones_mantenimiento, 1, if(ccomisiones_mantenimiento = avg3_ccomisiones_mantenimiento, 0, -1)) as ccomisiones_mantenimiento_comp
#     , if(mcomisiones_mantenimiento > avg3_mcomisiones_mantenimiento, 1, if(mcomisiones_mantenimiento = avg3_mcomisiones_mantenimiento, 0, -1)) as mcomisiones_mantenimiento_comp
#     , if(ccomisiones_otras > avg3_ccomisiones_otras, 1, if(ccomisiones_otras = avg3_ccomisiones_otras, 0, -1)) as ccomisiones_otras_comp
#     , if(mcomisiones_otras > avg3_mcomisiones_otras, 1, if(mcomisiones_otras = avg3_mcomisiones_otras, 0, -1)) as mcomisiones_otras_comp
#     , if(cforex > avg3_cforex, 1, if(cforex = avg3_cforex, 0, -1)) as cforex_comp
#     , if(cforex_buy > avg3_cforex_buy, 1, if(cforex_buy = avg3_cforex_buy, 0, -1)) as cforex_buy_comp
#     , if(mforex_buy > avg3_mforex_buy, 1, if(mforex_buy = avg3_mforex_buy, 0, -1)) as mforex_buy_comp
#     , if(cforex_sell > avg3_cforex_sell, 1, if(cforex_sell = avg3_cforex_sell, 0, -1)) as cforex_sell_comp
#     , if(mforex_sell > avg3_mforex_sell, 1, if(mforex_sell = avg3_mforex_sell, 0, -1)) as mforex_sell_comp
#     , if(ctransferencias_recibidas > avg3_ctransferencias_recibidas, 1, if(ctransferencias_recibidas = avg3_ctransferencias_recibidas, 0, -1)) as ctransferencias_recibidas_comp
#     , if(mtransferencias_recibidas > avg3_mtransferencias_recibidas, 1, if(mtransferencias_recibidas = avg3_mtransferencias_recibidas, 0, -1)) as mtransferencias_recibidas_comp
#     , if(ctransferencias_emitidas > avg3_ctransferencias_emitidas, 1, if(ctransferencias_emitidas = avg3_ctransferencias_emitidas, 0, -1)) as ctransferencias_emitidas_comp
#     , if(mtransferencias_emitidas > avg3_mtransferencias_emitidas, 1, if(mtransferencias_emitidas = avg3_mtransferencias_emitidas, 0, -1)) as mtransferencias_emitidas_comp
#     , if(cextraccion_autoservicio > avg3_cextraccion_autoservicio, 1, if(cextraccion_autoservicio = avg3_cextraccion_autoservicio, 0, -1)) as cextraccion_autoservicio_comp
#     , if(mextraccion_autoservicio > avg3_mextraccion_autoservicio, 1, if(mextraccion_autoservicio = avg3_mextraccion_autoservicio, 0, -1)) as mextraccion_autoservicio_comp
#     , if(ccheques_depositados > avg3_ccheques_depositados, 1, if(ccheques_depositados = avg3_ccheques_depositados, 0, -1)) as ccheques_depositados_comp
#     , if(mcheques_depositados > avg3_mcheques_depositados, 1, if(mcheques_depositados = avg3_mcheques_depositados, 0, -1)) as mcheques_depositados_comp
#     , if(ccheques_emitidos > avg3_ccheques_emitidos, 1, if(ccheques_emitidos = avg3_ccheques_emitidos, 0, -1)) as ccheques_emitidos_comp
#     , if(mcheques_emitidos > avg3_mcheques_emitidos, 1, if(mcheques_emitidos = avg3_mcheques_emitidos, 0, -1)) as mcheques_emitidos_comp
#     , if(ccheques_depositados_rechazados > avg3_ccheques_depositados_rechazados, 1, if(ccheques_depositados_rechazados = avg3_ccheques_depositados_rechazados, 0, -1)) as ccheques_depositados_rechazados_comp
#     , if(mcheques_depositados_rechazados > avg3_mcheques_depositados_rechazados, 1, if(mcheques_depositados_rechazados = avg3_mcheques_depositados_rechazados, 0, -1)) as mcheques_depositados_rechazados_comp
#     , if(ccheques_emitidos_rechazados > avg3_ccheques_emitidos_rechazados, 1, if(ccheques_emitidos_rechazados = avg3_ccheques_emitidos_rechazados, 0, -1)) as ccheques_emitidos_rechazados_comp
#     , if(mcheques_emitidos_rechazados > avg3_mcheques_emitidos_rechazados, 1, if(mcheques_emitidos_rechazados = avg3_mcheques_emitidos_rechazados, 0, -1)) as mcheques_emitidos_rechazados_comp
#     , if(ccallcenter_transacciones > avg3_ccallcenter_transacciones, 1, if(ccallcenter_transacciones = avg3_ccallcenter_transacciones, 0, -1)) as ccallcenter_transacciones_comp
#     , if(chomebanking_transacciones > avg3_chomebanking_transacciones, 1, if(chomebanking_transacciones = avg3_chomebanking_transacciones, 0, -1)) as chomebanking_transacciones_comp
#     , if(ccajas_transacciones > avg3_ccajas_transacciones, 1, if(ccajas_transacciones = avg3_ccajas_transacciones, 0, -1)) as ccajas_transacciones_comp
#     , if(ccajas_consultas > avg3_ccajas_consultas, 1, if(ccajas_consultas = avg3_ccajas_consultas, 0, -1)) as ccajas_consultas_comp
#     , if(ccajas_depositos > avg3_ccajas_depositos, 1, if(ccajas_depositos = avg3_ccajas_depositos, 0, -1)) as ccajas_depositos_comp
#     , if(ccajas_extracciones > avg3_ccajas_extracciones, 1, if(ccajas_extracciones = avg3_ccajas_extracciones, 0, -1)) as ccajas_extracciones_comp
#     , if(ccajas_otras > avg3_ccajas_otras, 1, if(ccajas_otras = avg3_ccajas_otras, 0, -1)) as ccajas_otras_comp
#     , if(catm_trx > avg3_catm_trx, 1, if(catm_trx = avg3_catm_trx, 0, -1)) as catm_trx_comp
#     , if(matm > avg3_matm, 1, if(matm = avg3_matm, 0, -1)) as matm_comp
#     , if(catm_trx_other > avg3_catm_trx_other, 1, if(catm_trx_other = avg3_catm_trx_other, 0, -1)) as catm_trx_other_comp
#     , if(matm_other > avg3_matm_other, 1, if(matm_other = avg3_matm_other, 0, -1)) as matm_other_comp
#     , if(ctrx_quarter > avg3_ctrx_quarter, 1, if(ctrx_quarter = avg3_ctrx_quarter, 0, -1)) as ctrx_quarter_comp
#     , if(cmobile_app_trx > avg3_cmobile_app_trx, 1, if(cmobile_app_trx = avg3_cmobile_app_trx, 0, -1)) as cmobile_app_trx_comp
#     , if(Master_mfinanciacion_limite > avg3_Master_mfinanciacion_limite, 1, if(Master_mfinanciacion_limite = avg3_Master_mfinanciacion_limite, 0, -1)) as Master_mfinanciacion_limite_comp
#     , if(Master_msaldototal > avg3_Master_msaldototal, 1, if(Master_msaldototal = avg3_Master_msaldototal, 0, -1)) as Master_msaldototal_comp
#     , if(Master_msaldopesos > avg3_Master_msaldopesos, 1, if(Master_msaldopesos = avg3_Master_msaldopesos, 0, -1)) as Master_msaldopesos_comp
#     , if(Master_msaldodolares > avg3_Master_msaldodolares, 1, if(Master_msaldodolares = avg3_Master_msaldodolares, 0, -1)) as Master_msaldodolares_comp
#     , if(Master_mconsumospesos > avg3_Master_mconsumospesos, 1, if(Master_mconsumospesos = avg3_Master_mconsumospesos, 0, -1)) as Master_mconsumospesos_comp
#     , if(Master_mconsumosdolares > avg3_Master_mconsumosdolares, 1, if(Master_mconsumosdolares = avg3_Master_mconsumosdolares, 0, -1)) as Master_mconsumosdolares_comp
#     , if(Master_mlimitecompra > avg3_Master_mlimitecompra, 1, if(Master_mlimitecompra = avg3_Master_mlimitecompra, 0, -1)) as Master_mlimitecompra_comp
#     , if(Master_madelantopesos > avg3_Master_madelantopesos, 1, if(Master_madelantopesos = avg3_Master_madelantopesos, 0, -1)) as Master_madelantopesos_comp
#     , if(Master_madelantodolares > avg3_Master_madelantodolares, 1, if(Master_madelantodolares = avg3_Master_madelantodolares, 0, -1)) as Master_madelantodolares_comp
#     , if(Master_mpagado > avg3_Master_mpagado, 1, if(Master_mpagado = avg3_Master_mpagado, 0, -1)) as Master_mpagado_comp
#     , if(Master_mpagospesos > avg3_Master_mpagospesos, 1, if(Master_mpagospesos = avg3_Master_mpagospesos, 0, -1)) as Master_mpagospesos_comp
#     , if(Master_mpagosdolares > avg3_Master_mpagosdolares, 1, if(Master_mpagosdolares = avg3_Master_mpagosdolares, 0, -1)) as Master_mpagosdolares_comp
#     , if(Master_mconsumototal > avg3_Master_mconsumototal, 1, if(Master_mconsumototal = avg3_Master_mconsumototal, 0, -1)) as Master_mconsumototal_comp
#     , if(Master_cconsumos > avg3_Master_cconsumos, 1, if(Master_cconsumos = avg3_Master_cconsumos, 0, -1)) as Master_cconsumos_comp
#     , if(Master_cadelantosefectivo > avg3_Master_cadelantosefectivo, 1, if(Master_cadelantosefectivo = avg3_Master_cadelantosefectivo, 0, -1)) as Master_cadelantosefectivo_comp
#     , if(Master_mpagominimo > avg3_Master_mpagominimo, 1, if(Master_mpagominimo = avg3_Master_mpagominimo, 0, -1)) as Master_mpagominimo_comp
#     , if(Visa_mfinanciacion_limite > avg3_Visa_mfinanciacion_limite, 1, if(Visa_mfinanciacion_limite = avg3_Visa_mfinanciacion_limite, 0, -1)) as Visa_mfinanciacion_limite_comp
#     , if(Visa_msaldototal > avg3_Visa_msaldototal, 1, if(Visa_msaldototal = avg3_Visa_msaldototal, 0, -1)) as Visa_msaldototal_comp
#     , if(Visa_msaldopesos > avg3_Visa_msaldopesos, 1, if(Visa_msaldopesos = avg3_Visa_msaldopesos, 0, -1)) as Visa_msaldopesos_comp
#     , if(Visa_msaldodolares > avg3_Visa_msaldodolares, 1, if(Visa_msaldodolares = avg3_Visa_msaldodolares, 0, -1)) as Visa_msaldodolares_comp
#     , if(Visa_mconsumospesos > avg3_Visa_mconsumospesos, 1, if(Visa_mconsumospesos = avg3_Visa_mconsumospesos, 0, -1)) as Visa_mconsumospesos_comp
#     , if(Visa_mconsumosdolares > avg3_Visa_mconsumosdolares, 1, if(Visa_mconsumosdolares = avg3_Visa_mconsumosdolares, 0, -1)) as Visa_mconsumosdolares_comp
#     , if(Visa_mlimitecompra > avg3_Visa_mlimitecompra, 1, if(Visa_mlimitecompra = avg3_Visa_mlimitecompra, 0, -1)) as Visa_mlimitecompra_comp
#     , if(Visa_madelantopesos > avg3_Visa_madelantopesos, 1, if(Visa_madelantopesos = avg3_Visa_madelantopesos, 0, -1)) as Visa_madelantopesos_comp
#     , if(Visa_madelantodolares > avg3_Visa_madelantodolares, 1, if(Visa_madelantodolares = avg3_Visa_madelantodolares, 0, -1)) as Visa_madelantodolares_comp
#     , if(Visa_mpagado > avg3_Visa_mpagado, 1, if(Visa_mpagado = avg3_Visa_mpagado, 0, -1)) as Visa_mpagado_comp
#     , if(Visa_mpagospesos > avg3_Visa_mpagospesos, 1, if(Visa_mpagospesos = avg3_Visa_mpagospesos, 0, -1)) as Visa_mpagospesos_comp
#     , if(Visa_mpagosdolares > avg3_Visa_mpagosdolares, 1, if(Visa_mpagosdolares = avg3_Visa_mpagosdolares, 0, -1)) as Visa_mpagosdolares_comp
#     , if(Visa_mconsumototal > avg3_Visa_mconsumototal, 1, if(Visa_mconsumototal = avg3_Visa_mconsumototal, 0, -1)) as Visa_mconsumototal_comp
#     , if(Visa_cconsumos > avg3_Visa_cconsumos, 1, if(Visa_cconsumos = avg3_Visa_cconsumos, 0, -1)) as Visa_cconsumos_comp
#     , if(Visa_cadelantosefectivo > avg3_Visa_cadelantosefectivo, 1, if(Visa_cadelantosefectivo = avg3_Visa_cadelantosefectivo, 0, -1)) as Visa_cadelantosefectivo_comp
#     , if(Visa_mpagominimo > avg3_Visa_mpagominimo, 1, if(Visa_mpagominimo = avg3_Visa_mpagominimo, 0, -1)) as Visa_mpagominimo_comp
#     , if(map_margen > avg3_map_margen, 1, if(map_margen = avg3_map_margen, 0, -1)) as map_margen_comp
#     , if(cc_ca > avg3_cc_ca, 1, if(cc_ca = avg3_cc_ca, 0, -1)) as cc_ca_comp
#     , if(ctarjeta_total > avg3_ctarjeta_total, 1, if(ctarjeta_total = avg3_ctarjeta_total, 0, -1)) as ctarjeta_total_comp
#     , if(ctarjeta_total_transacciones > avg3_ctarjeta_total_transacciones, 1, if(ctarjeta_total_transacciones = avg3_ctarjeta_total_transacciones, 0, -1)) as ctarjeta_total_transacciones_comp
#     , if(mtarjeta_total_consumo > avg3_mtarjeta_total_consumo, 1, if(mtarjeta_total_consumo = avg3_mtarjeta_total_consumo, 0, -1)) as mtarjeta_total_consumo_comp
#     , if(mplazo_fijo > avg3_mplazo_fijo, 1, if(mplazo_fijo = avg3_mplazo_fijo, 0, -1)) as mplazo_fijo_comp
#     , if(minversion1 > avg3_minversion1, 1, if(minversion1 = avg3_minversion1, 0, -1)) as minversion1_comp
#     , if(cseguro_todos > avg3_cseguro_todos, 1, if(cseguro_todos = avg3_cseguro_todos, 0, -1)) as cseguro_todos_comp
#     , if(mpayroll_todos > avg3_mpayroll_todos, 1, if(mpayroll_todos = avg3_mpayroll_todos, 0, -1)) as mpayroll_todos_comp
#     , if(consumos_mas_DA > avg3_consumos_mas_DA, 1, if(consumos_mas_DA = avg3_consumos_mas_DA, 0, -1)) as consumos_mas_DA_comp
#     , if(ctarjeta_todas_debitos_automaticos > avg3_ctarjeta_todas_debitos_automaticos, 1, if(ctarjeta_todas_debitos_automaticos = avg3_ctarjeta_todas_debitos_automaticos, 0, -1)) as ctarjeta_todas_debitos_automaticos_comp
#     , if(mtarjeta_todas_debitos_automaticos > avg3_mtarjeta_todas_debitos_automaticos, 1, if(mtarjeta_todas_debitos_automaticos = avg3_mtarjeta_todas_debitos_automaticos, 0, -1)) as mtarjeta_todas_debitos_automaticos_comp
#     , if(ctarjeta_todas_descuentos > avg3_ctarjeta_todas_descuentos, 1, if(ctarjeta_todas_descuentos = avg3_ctarjeta_todas_descuentos, 0, -1)) as ctarjeta_todas_descuentos_comp
#     , if(mtarjeta_todas_descuentos > avg3_mtarjeta_todas_descuentos, 1, if(mtarjeta_todas_descuentos = avg3_mtarjeta_todas_descuentos, 0, -1)) as mtarjeta_todas_descuentos_comp
#     , if(comisiones > avg3_comisiones, 1, if(comisiones = avg3_comisiones, 0, -1)) as comisiones_comp
#     , if(mcomisiones_suma > avg3_mcomisiones_suma, 1, if(mcomisiones_suma = avg3_mcomisiones_suma, 0, -1)) as mcomisiones_suma_comp
#     , if(ctransferencias > avg3_ctransferencias, 1, if(ctransferencias = avg3_ctransferencias, 0, -1)) as ctransferencias_comp
#     , if(mtransferencias > avg3_mtransferencias, 1, if(mtransferencias = avg3_mtransferencias, 0, -1)) as mtransferencias_comp
#     , if(tarjeta_msaldototal > avg3_tarjeta_msaldototal, 1, if(tarjeta_msaldototal = avg3_tarjeta_msaldototal, 0, -1)) as tarjeta_msaldototal_comp
#     , if(tarjetas_msaldopesos > avg3_tarjetas_msaldopesos, 1, if(tarjetas_msaldopesos = avg3_tarjetas_msaldopesos, 0, -1)) as tarjetas_msaldopesos_comp
#     , if(tarjetas_msaldodolares > avg3_tarjetas_msaldodolares, 1, if(tarjetas_msaldodolares = avg3_tarjetas_msaldodolares, 0, -1)) as tarjetas_msaldodolares_comp
#     , if(tarjetas_mconsumospesos > avg3_tarjetas_mconsumospesos, 1, if(tarjetas_mconsumospesos = avg3_tarjetas_mconsumospesos, 0, -1)) as tarjetas_mconsumospesos_comp
#     , if(tarjetas_mconsumosdolares > avg3_tarjetas_mconsumosdolares, 1, if(tarjetas_mconsumosdolares = avg3_tarjetas_mconsumosdolares, 0, -1)) as tarjetas_mconsumosdolares_comp
#     , if(tarjetas_mlimitecompra > avg3_tarjetas_mlimitecompra, 1, if(tarjetas_mlimitecompra = avg3_tarjetas_mlimitecompra, 0, -1)) as tarjetas_mlimitecompra_comp
#     , if(tarjetas_madelantopesos > avg3_tarjetas_madelantopesos, 1, if(tarjetas_madelantopesos = avg3_tarjetas_madelantopesos, 0, -1)) as tarjetas_madelantopesos_comp
#     , if(tarjetas_madelantodolares > avg3_tarjetas_madelantodolares, 1, if(tarjetas_madelantodolares = avg3_tarjetas_madelantodolares, 0, -1)) as tarjetas_madelantodolares_comp
#     , if(tarjetas_mpagado > avg3_tarjetas_mpagado, 1, if(tarjetas_mpagado = avg3_tarjetas_mpagado, 0, -1)) as tarjetas_mpagado_comp
#     , if(tarjetas_mpagospesos > avg3_tarjetas_mpagospesos, 1, if(tarjetas_mpagospesos = avg3_tarjetas_mpagospesos, 0, -1)) as tarjetas_mpagospesos_comp
#     , if(tarjetas_mpagosdolares > avg3_tarjetas_mpagosdolares, 1, if(tarjetas_mpagosdolares = avg3_tarjetas_mpagosdolares, 0, -1)) as tarjetas_mpagosdolares_comp
#     , if(tarjetas_mconsumototal > avg3_tarjetas_mconsumototal, 1, if(tarjetas_mconsumototal = avg3_tarjetas_mconsumototal, 0, -1)) as tarjetas_mconsumototal_comp
#     , if(tarjetas_cconsumos > avg3_tarjetas_cconsumos, 1, if(tarjetas_cconsumos = avg3_tarjetas_cconsumos, 0, -1)) as tarjetas_cconsumos_comp
#     , if(tarjetas_cadelantosefectivo > avg3_tarjetas_cadelantosefectivo, 1, if(tarjetas_cadelantosefectivo = avg3_tarjetas_cadelantosefectivo, 0, -1)) as tarjetas_cadelantosefectivo_comp
#     , if(tarjetas_mpagominimo > avg3_tarjetas_mpagominimo, 1, if(tarjetas_mpagominimo = avg3_tarjetas_mpagominimo, 0, -1)) as tarjetas_mpagominimo_comp
#     , if(tarjetas_mfinanciacion_limite > avg3_tarjetas_mfinanciacion_limite, 1, if(tarjetas_mfinanciacion_limite = avg3_tarjetas_mfinanciacion_limite, 0, -1)) as tarjetas_mfinanciacion_limite_comp
#     , if(matm_total > avg3_matm_total, 1, if(matm_total = avg3_matm_total, 0, -1)) as matm_total_comp
# from competencia_01

### Otras variables

In [19]:
# %%sql
# create or replace table competencia_01 as
# select
#     *
#     , if(tcallcenter = 1 or thomebanking = 1, 1,0) as tchannel
#     , if(Master_delinquency = 1 or Visa_delinquency = 1, 1,0) as any_delinquency
#     , ifnull(Master_delinquency, 0) +  ifnull(Visa_delinquency, 0) as total_delinquency
#     , CASE WHEN Master_status = 0 THEN 0
#         WHEN Master_status = 6 THEN 15
#         WHEN Master_status = 7 THEN 20
#         WHEN Master_status = 9 THEN 5 END AS Master_status_v2
#     , CASE WHEN Visa_status = 0 THEN 0
#         WHEN Visa_status = 6 THEN 15
#         WHEN Visa_status = 7 THEN 20
#         WHEN Visa_status = 9 THEN 5 END AS Visa_status_v2
# from competencia_01

### Deciles

In [20]:
# %%sql
# create or replace table competencia_01 as
# select
#     *
#     , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5
# from competencia_01

### Percent ranks

In [21]:
# %%sql
# create or replace table competencia_01 as
# select
#     *
#     , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3
# from competencia_01

### Deltas

In [22]:
# Parecido a slope, pero viendo la variacion con el lag anterior

### Guardar datos

In [23]:
%sql COPY competencia_01 TO 'C:/Users/Cristian Burich/Desktop/MA/segundo/eyf/datasets/competencia_01_fe_v02.csv' (FORMAT CSV, HEADER TRUE);

# if you want to use dataset_path you need to build the path in another line because once you use %sql or %%sql you can't use a parameter
# outside the function.


,Success


In [28]:
%%sql 

select * from competencia_01

limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,slope_Visa_mlimitecompra,slope_Visa_madelantopesos,slope_Visa_madelantodolares,slope_Visa_mpagado,slope_Visa_mpagospesos,slope_Visa_mpagosdolares,slope_Visa_mconsumototal,slope_Visa_cconsumos,slope_Visa_cadelantosefectivo,slope_Visa_mpagominimo
0,249228180,202101,1,0,0,66,320,175.87,2600.67,276.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249228180,202102,1,0,0,66,321,30.95,2752.73,160.05,...,0.0,0.0,0.0,0.0,3149.500,0.0,12390.460,2.000000e+00,0.0,868.020
2,249228180,202103,1,0,0,66,322,-4.12,2546.75,192.05,...,0.0,0.0,0.0,0.0,-4804.020,0.0,3051.325,-5.000000e-01,0.0,211.140
3,249228180,202104,1,0,0,66,323,214.11,2406.24,309.60,...,0.0,0.0,0.0,0.0,-2939.535,0.0,-3150.095,-1.000000e+00,0.0,-199.410
4,249228180,202105,1,0,0,66,324,-22.75,2046.74,118.20,...,0.0,0.0,0.0,0.0,3237.480,0.0,1007.905,5.000000e-01,0.0,199.410
5,249228180,202106,1,0,0,66,325,-138.82,1445.30,417.78,...,0.0,0.0,0.0,0.0,-1987.065,0.0,3444.075,0.000000e+00,0.0,416.415
6,249228180,202107,1,0,0,66,326,368.95,1292.44,253.83,...,0.0,0.0,0.0,0.0,-4297.285,0.0,4163.490,1.000000e+00,0.0,228.735
7,249228180,202108,1,0,0,66,327,1915.94,2691.40,2190.03,...,0.0,0.0,0.0,0.0,-4165.905,0.0,1583.995,-2.220446e-16,0.0,-17.595
8,249255311,202101,1,0,0,55,196,14740.52,78936.06,421.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,249255311,202102,1,0,0,55,197,13227.81,90657.96,329.49,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000
